#### Simple CNN for image classification

In [1]:
import torch
from torch import nn

In [6]:
model = nn.Sequential()
img_channels = 3
model.add_module(name = 'conv0', module = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, padding='same'))



In [4]:
naive_input = torch.ones(size=(32, 3, 256, 256))
naive_input.shape

torch.Size([32, 3, 256, 256])

In [8]:
model(naive_input).shape

torch.Size([32, 12, 256, 256])